In [ ]:
%%writefile test_cuda.cu
#include <stdio.h>
#include <assert.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void test(){
    printf("Hi Cuda World\n");
}

int main( int argc, char** argv )
{
    test<<<1,8>>>();
    cudaDeviceSynchronize();
    return 0;
}


Writing test_cuda.cu


In [ ]:
!nvcc test_cuda.cu

In [ ]:
!./a.out

Hi Cuda World
Hi Cuda World
Hi Cuda World
Hi Cuda World
Hi Cuda World
Hi Cuda World
Hi Cuda World
Hi Cuda World
